In [45]:
import rasterio
import numpy as np
import seaborn as sns

In [46]:
maxprecep=rasterio.open("Data/PrecipitationHistory/CHRIPS_Max_NP_40.tif").read()
meanprecep=rasterio.open("Data/PrecipitationHistory/CHRIPS_Mean_NP_40.tif").read()
stdprecep=rasterio.open("Data/PrecipitationHistory/CHRIPS_STD_NP_40.tif").read()
arg_sorted=np.argsort(maxprecep,axis=0)[::-1]
maxprecep_sorted=np.take_along_axis(maxprecep, arg_sorted, 0)
meanprecep_sorted=np.take_along_axis(maxprecep, arg_sorted, 0)
stdprecep_sorted=np.take_along_axis(maxprecep, arg_sorted, 0)

In [47]:
# for given 40 years of data the following ranks should represent the return periods of 5, 10, 15, 20 years
five_return=round(41/5)
ten_return=round(41/10)
fifteen_return=round(41/15)
twenty_return=round(41/20)

In [66]:
src=rasterio.open("Data/PrecipitationHistory/CHRIPS_Max_NP_40.tif")
def savefile(array,filename):
    with rasterio.Env():
        profile = src.profile
        profile.update(
            dtype=rasterio.float32,
            count=1,
            compress='lzw')
        with rasterio.open(filename, 'w', **profile) as dst:
            dst.write(array, 1)

In [67]:
def save_rp_data(rp):
    return_rank=round(41/rp)
    print(return_rank)
    mx_design=maxprecep_sorted[return_rank]
    me_design=meanprecep_sorted[return_rank]
    st_design=stdprecep_sorted[return_rank]
    savefile(mx_design,f"Data/DesignRainfall/mx_design_{rp}.tif")
    savefile(me_design,f"Data/DesignRainfall/me_design_{rp}.tif")
    savefile(st_design,f"Data/DesignRainfall/st_design_{rp}.tif")
    

In [68]:
for rp in [5,10,15,20]:
    save_rp_data(rp)

8
4
3
2
